In [25]:
import numpy as np
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import pyLDAvis
import pyLDAvis.sklearn
from preprocess import lemm_string

pyLDAvis.enable_notebook()

# load and drop NaNs
train = pd.read_excel('../../tweets/csv/train.xls', header=None)
train.dropna(inplace=True)
train[0] = train[0].map(lemm_string)

test = pd.read_excel('../../tweets/csv/test1.xls')
test.dropna(inplace=True)
test['tweets'] = test['tweets'].map(lemm_string)

# segment data by company
# Starbucks
sbux_train = train[train[0].str.contains('starbucks')]
sbux_test = test[test['tweets'].str.contains('starbucks')]
# Chipotle
cmg_train = train[train[0].str.contains('chipotle')]
cmg_test = test[test['tweets'].str.contains('chipotle')]
# McDonalds
mcd_train = train[train[0].str.contains('mcdonalds')]
mcd_test = test[test['tweets'].str.contains('mcdonalds')]

n_features = 500
ngrams = [1,2]
max_df = .5
n_topics = 8
n_top_words = 10


def print_top_words(model, feature_names, n_top_words):
    words = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % (topic_idx + 1))
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))


def tfidf(train, test):
    tf = CountVectorizer(stop_words=ENGLISH_STOP_WORDS, max_features=n_features, ngram_range=ngrams, max_df=.5)
    train = tf.fit_transform(train)
    feature_names = tf.get_feature_names()
    test = tf.transform(test)
    return train, test, feature_names, tf


def decompose(model, train, test, tf):
    feature_names = tf.get_feature_names()
    model.fit(train)
    print_top_words(model, feature_names, n_top_words)
    train_dist_preds = model.transform(train)
    train_preds = np.argmax(train_dist_preds, axis=1)
    test_dist_preds = model.transform(test)
    test_preds = np.argmax(test_dist_preds, axis=1)
    return train_preds, test_preds, model

def topic_summaries(df, test_preds, mod_name):
    df['topics'] = test_preds.tolist()
    df['topics'] = df['topics'] + 1
    file_dir = '../../tweets/topic_dfs/{}_topic_preds.csv'.format(mod_name)
    df.to_csv(file_dir)
    topic_df = pd.read_csv(file_dir)
    summary = topic_df.groupby('topics')['labels'].mean()
    count = topic_df.groupby(['topics','labels'])['tweets'].count()
    print summary
    return summary, count

In [26]:
# UPDATE COMPANY HERE
train =  cmg_train.values.reshape(cmg_train.values.shape[0],)
# UPDATE COMPANY HERE
test = cmg_test['tweets'].values.reshape(cmg_test.values.shape[0],)
train, test, feature_names, tf = tfidf(train, test)
lda = LatentDirichletAllocation(n_topics=n_topics, learning_method='online', learning_offset=50.)
nmf = NMF(n_components=n_topics, init='random')
lsa = TruncatedSVD(n_components=n_topics, algorithm='randomized')
###############################################
model = lda # Set model here
mod_name = 'lda'
###############################################
train_preds, test_preds, model = decompose(model, train, test, tf)
# UPDATE COMPANY HERE
summary, count = topic_summaries(cmg_test, test_preds, mod_name=mod_name)

Topic #1:
burrito, food, right, chipotle burrito, youre, white, good, ass, line, sounds
Topic #2:
like, im, day, love, need, eating, valentines, craving, ive, craving chipotle
Topic #3:
sob, people, really, thinkingface, extra, order, im, thats, lunch, gonna
Topic #4:
free, bowl, time, free chipotle, know, burrito, mexican, ill, fundraiser, food
Topic #5:
eat, chipotletweets, eat chipotle, good, hearteyes, lol, burritos, come, day, skintone3
Topic #6:
want, want chipotle, joy, just, got, jacksfilms, im, support, think, joy joy
Topic #7:
dont, better, life, yum, work, make, tomorrow, buy, bring, best
Topic #8:
today, chicken, weary, taco, bad, chipotle today, sauce, new, cheese, ate
topics
1    2.160714
2    2.396825
3    2.152542
4    2.311111
5    2.479167
6    2.698630
7    2.266667
8    2.204082
Name: labels, dtype: float64


In [27]:
print count


topics  labels
1       1.0       15
        2.0       17
        3.0       24
2       1.0       13
        2.0       12
        3.0       38
3       1.0       18
        2.0       14
        3.0       27
4       1.0        8
        2.0       15
        3.0       22
5       1.0       10
        2.0        5
        3.0       33
6       1.0        5
        2.0       12
        3.0       56
7       1.0       10
        2.0       13
        3.0       22
8       1.0       13
        2.0       13
        3.0       23
Name: tweets, dtype: int64


In [24]:
pyLDAvis.sklearn.prepare(model, train, tf, R=20)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      15.174356        1       1 -0.215792 -0.023250
7      13.841694        1       2  0.147130  0.252520
2      13.341833        1       3 -0.201360 -0.150268
4      12.247336        1       4 -0.047821  0.177238
3      12.061517        1       5  0.012694 -0.204879
1      11.794057        1       6  0.240239 -0.255163
5      11.092005        1       7 -0.168903  0.118455
0      10.447202        1       8  0.233813  0.085348, topic_info=     Category        Freq                 Term       Total  loglift  logprob
term                                                                        
460   Default  900.000000                 want  900.000000  20.0000  20.0000
47    Default  710.000000              burrito  710.000000  19.0000  19.0000
135   Default  560.000000                  day  560.000000  18.0000  18.0000
146   Default  555.000000                 dont  555.000000  17.0000  17.0000
461   Default  592.000000        want chipotle  592.000000  16.0000  16.0000
254   Default  551.000000                  joy  551.000000  15.0000  15.0000
151   Default  507.000000                  eat  507.000000  14.0000  14.0000
199   Default  483.000000                  got  483.000000  13.0000  13.0000
256   Default  855.000000                 just  855.000000  12.0000  12.0000
430   Default  436.000000                today  436.000000  11.0000  11.0000
242   Default  997.000000                   im  997.000000  10.0000  10.0000
287   Default  449.000000                 love  449.000000   9.0000   9.0000
39    Default  367.000000                 bowl  367.000000   8.0000   8.0000
170   Default  412.000000                 food  412.000000   7.0000   7.0000
173   Default  508.000000                 free  508.000000   6.0000   6.0000
274   Default  710.000000                 like  710.000000   5.0000   5.0000
314   Default  308.000000                 need  308.000000   4.0000   4.0000
352   Default  350.000000               really  350.000000   3.0000   3.0000
206   Default  367.000000                 guac  367.000000   2.0000   2.0000
61    Default  338.000000              chicken  338.000000   1.0000   1.0000
287    Topic1  448.794632                 love  449.680034   1.8836  -2.8415
206    Topic1  366.139312                 guac  367.011983   1.8832  -3.0451
153    Topic1  269.384922               eating  270.257682   1.8823  -3.3520
50     Topic1  195.948403             burritos  196.821265   1.8811  -3.6702
195    Topic1  191.347202                going  192.219888   1.8810  -3.6940
288    Topic1  171.665681        love chipotle  172.538069   1.8805  -3.8025
154    Topic1  166.723487      eating chipotle  167.595919   1.8803  -3.8318
303    Topic1  166.140103              mexican  167.012380   1.8803  -3.8353
275    Topic1  151.975776        like chipotle  152.848272   1.8798  -3.9244
365    Topic1  139.151106                sauce  140.023826   1.8793  -4.0125
...       ...         ...                  ...         ...      ...      ...
119    Topic7   82.542785                class   83.409609   2.1885  -4.2214
230    Topic7   74.352615                  hit   75.219129   2.1874  -4.3259
40     Topic7   71.464740        bowl chipotle   72.330978   2.1869  -4.3655
225    Topic7   70.660399  hearteyes hearteyes   71.526546   2.1868  -4.3768
396    Topic7   70.463610       stuckouttongue   71.330027   2.1867  -4.3796
256    Topic7  432.619133                 just  855.890251   1.5167  -2.5649
242    Topic7  446.978197                   im  997.914893   1.3958  -2.5322
274    Topic7   94.176214                 like  710.873608   0.1776  -4.0895
314    Topic8  307.177623                 need  308.043414   2.2560  -2.8474
327    Topic8  184.725639                order  185.591560   2.2542  -3.3560
413    Topic8  166.888909                thats  167.754957   2.2537  -3.4575
317    Topic8  146.697749    